In [ ]:
### 이메일 뉴스알람 프로젝트
# 준비사항 : 로컬 환경에서 아웃룩이 열려있어야 합니다.
# 질의응답 : https://chatgpt.com/share/22b4c928-5371-4bc7-90a0-391b3700cb1e

mail_subject = input("메일제목을 적어주세요:")
your_mail = input("수신자 메일을 적어주세요:")
query_info = input("원하는 키워드를 적어주세요:")

#1. 크롤링
# 뉴스 기사를 요약하여 메일로 전송하는 프로그램
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
delay_second = 1
time.sleep(delay_second)

# 1-1.인터넷 브라우져 열기
!pip install selenium==4.1.5
from selenium import webdriver
driver = webdriver.Chrome("c:/chromedriver.exe")
time.sleep(delay_second)

# 1-2. 원하는 사이트 접속하기 : 네이버 - 뉴스사이트
news_url = "https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query="
# query_info = "SK텔레콤"
driver.get(news_url + query_info)
time.sleep(delay_second)

# 1-3. 마우스 아래로 스크롤하기 5번
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(driver)
time.sleep(delay_second)
for _ in range(5):
    actions.send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(0.1)  # 스크롤 후 잠시 대기
time.sleep(delay_second)

#1-4. 뉴스 정보를 찾아서 리스트에 넣기
sel_01 = driver.find_elements(By.CLASS_NAME,'news_area')
news_info = []
for a in sel_01:
    time = ""
    target = False  # 원하는 대상인지 체크하기 위한 변수,
    title =a.find_element(By.CLASS_NAME,'news_tit').text
    # 네이버 뉴스의 간략한 내용 정리를 description에 넣기
    description = a.find_element(By.CLASS_NAME,'news_dsc').text
    url = a.find_element(By.CLASS_NAME,'news_tit').get_attribute('href')
    time_sel = a.find_elements(By.CLASS_NAME,'info')
    if len(time_sel)==4: # time_sel 이 4일때는 2번 인덱스, 나머지는 1번인덱스
        time = time_sel[2].text
    else:
        time = time_sel[1].text
    news_info.append([title, description, url, time, target])
# ============================================

#1-6. 수집된 뉴스에 대해서 원하는 날짜이내의 뉴스만 target 값을 True로 변경
from datetime import datetime, timedelta
# 기본 시간 제한 (일)  :  지정된 일자 이내의 뉴스만 유효하게 관리
default_time = 2
# 현재 시간
now = datetime.now()
# 시간 파싱 및 day_TF 갱신 로직
for news in news_info:
    time_str = news[3]
    time_ago = 0
    if "시간" in time_str:
        hours_ago = int(time_str.split("시간 전")[0].strip())
        time_ago = timedelta(hours=hours_ago)
    elif "분" in time_str:
        minutes_ago = int(time_str.split("분 전")[0].strip())
        time_ago = timedelta(minutes=minutes_ago)
    elif "일" in time_str:
        days_ago = int(time_str.split("일 전")[0].strip())
        time_ago = timedelta(days=days_ago)
    elif "주" in time_str:
        weeks_ago = int(time_str.split("주 전")[0].strip())
        time_ago = timedelta(weeks=weeks_ago)
    # 날짜 계산
    news_time = now - time_ago
    # 기본 시간 제한 내에 해당하면 target를 True로 설정
    if news_time >= now - timedelta(days=default_time):
        news[4] = True
# 결과 확인
for news in news_info:
    if(news[4] == True):
        print(news)
driver.close()

#2. html 로 구성
import textwrap

html_outputs = "" #기사를 html로 만들어야 하나끼나, 하나씩 쌓아나가 봅시다.
count = 0 #뉴스기사의 카운팅을 시작해 봅시다.

for news in news_info:
    count += 1
    title = news[0]  # 제목
    summary = news[1]  # 요약
    url = news[2]  # URL
    date = str(now.date())+"으로부터 "+news[3] # 작성일

    # 본문 줄바꿈 처리 (50자 기준)
    wrapped_summary = "<br>".join(textwrap.wrap(summary, width=50)) #chatgpt가 도와준 코드입니다.

    html_output = f"""
    <tr style="height:16.5pt">
        <td width="18" nowrap="" style="width:10.8pt;padding:0cm 4.95pt 0cm 4.95pt;height:16.5pt">
            <p class="MsoNormal" align="left" style="text-align:left;text-autospace:ideograph-other;word-break:keep-all">
                <b><span lang="EN-US" style="font-size:11.0pt;color:black">{count}.<u></u><u></u></span></b>
            </p>
        </td>
        <td width="1015" nowrap="" style="width:609.1pt;padding:0cm 4.95pt 0cm 4.95pt;height:16.5pt">
            <p class="MsoNormal" align="left" style="text-align:left;text-autospace:ideograph-other;word-break:keep-all">
                <b><span lang="EN-US" style="font-size:11.0pt;color:black">{title} ({date})</span></b><br>
                <span style="font-size:11.0pt;color:black">&nbsp;</span><br>
                <span style="font-size:11.0pt;color:black">{wrapped_summary}</span><br>
                <a href="{url}" style="font-size:11.0pt;color:blue;text-decoration:underline;">{url}</a>
            </p>
        </td>
    </tr>
    <hr style="border:1px solid #000; margin:20px 0;">
    """

    html_outputs += html_output #쌓아나가야죠 기사들을.

my_html = html_outputs #완성된 html을 본문으로!


#3. outlook 메일 전송
#Outlook 로그인 -개별설치
#메일작성하기 (텍스트보내기)
import win32com.client

outlook=win32com.client.Dispatch("Outlook.Application")
Txoutlook = outlook.CreateItem(0)
Txoutlook.To = your_mail #개인 이메일 작성
Txoutlook.CC = ""#개인 이메일 작성
Txoutlook.Subject = mail_subject
Txoutlook.HTMLBody = my_html

#Txoutlook.Display(True)
try:
    Txoutlook.send()
except:
    print('')

## 코드 퍼포먼스 증가 (selenium --> requests)
# 휠기능 제외


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import textwrap
import win32com.client

# 사용자 입력
mail_subject = input("메일제목을 적어주세요: ")
your_mail = input("수신자 메일을 적어주세요: ")
query_info = input("원하는 키워드를 적어주세요: ")

# 1. 크롤링
news_url = f"https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={query_info}"
response = requests.get(news_url)
soup = BeautifulSoup(response.text, 'html.parser')

# 1-1. 뉴스 정보를 찾아서 리스트에 넣기
news_info = []
articles = soup.select('.news_area')

for article in articles:
    title = article.select_one('.news_tit').get_text()
    description = article.select_one('.news_dsc').get_text()
    url = article.select_one('.news_tit')['href']
    time_elements = article.select('.info')

    # 뉴스의 시간이 여러 가지 형태로 존재할 수 있음
    time = time_elements[2].get_text() if len(time_elements) == 4 else time_elements[1].get_text()
    news_info.append([title, description, url, time, False])

# 1-2. 수집된 뉴스에 대해서 원하는 날짜 이내의 뉴스만 target 값을 True로 변경
now = datetime.now()
default_time = 2

for news in news_info:
    time_str = news[3]
    time_ago = timedelta()
    if "시간" in time_str:
        time_ago = timedelta(hours=int(time_str.split("시간 전")[0].strip()))
    elif "분" in time_str:
        time_ago = timedelta(minutes=int(time_str.split("분 전")[0].strip()))
    elif "일" in time_str:
        time_ago = timedelta(days=int(time_str.split("일 전")[0].strip()))
    elif "주" in time_str:
        time_ago = timedelta(weeks=int(time_str.split("주 전")[0].strip()))

    news_time = now - time_ago
    if news_time >= now - timedelta(days=default_time):
        news[4] = True

# 결과 확인
for news in news_info:
    if news[4]:
        print(news)

# 2. HTML로 구성
html_outputs = ""
for count, news in enumerate(news_info, 1):
    if news[4]:
        wrapped_summary = "<br>".join(textwrap.wrap(news[1], width=50))
        html_output = f"""
        <tr style="height:16.5pt">
            <td width="18" nowrap="" style="width:10.8pt;padding:0cm 4.95pt 0cm 4.95pt;height:16.5pt">
                <p class="MsoNormal" align="left" style="text-align:left;text-autospace:ideograph-other;word-break:keep-all">
                    <b><span lang="EN-US" style="font-size:11.0pt;color:black">{count}.<u></u><u></u></span></b>
                </p>
            </td>
            <td width="1015" nowrap="" style="width:609.1pt;padding:0cm 4.95pt 0cm 4.95pt;height:16.5pt">
                <p class="MsoNormal" align="left" style="text-align:left;text-autospace:ideograph-other;word-break:keep-all">
                    <b><span lang="EN-US" style="font-size:11.0pt;color:black">{news[0]} ({now.date()}으로부터 {news[3]})</span></b><br>
                    <span style="font-size:11.0pt;color:black">&nbsp;</span><br>
                    <span style="font-size:11.0pt;color:black">{wrapped_summary}</span><br>
                    <a href="{news[2]}" style="font-size:11.0pt;color:blue;text-decoration:underline;">{news[2]}</a>
                </p>
            </td>
        </tr>
        <hr style="border:1px solid #000; margin:20px 0;">
        """
        html_outputs += html_output

# 3. Outlook 메일 전송
outlook = win32com.client.Dispatch("Outlook.Application")
Txoutlook = outlook.CreateItem(0)
Txoutlook.To = your_mail
Txoutlook.Subject = mail_subject
Txoutlook.HTMLBody = html_outputs

try:
    Txoutlook.Send()
except Exception as e:
    print(f"Error sending email: {e}")
